# はじめに

私たちは最近、新しいオープンソースの**Agents SDK**を発表しました。これは、軽量で使いやすく、最小限の抽象化を持つパッケージを使用して、エージェント型AIアプリケーションの構築を支援するために設計されています。

このクックブックでは、Agents SDKをStripeのAPIと組み合わせて活用し、多くの企業が直面する一般的な運用課題である紛争管理を処理する方法を実演します。具体的には、以下の2つの実世界のシナリオに焦点を当てます：

1. **企業のミス:**  
   注文の履行に失敗するなど、企業が明らかにエラーを犯したシナリオで、紛争を受け入れることが適切な対応となる場合。

2. **顧客の紛争（最終販売）:**  
   顧客が正しい商品を受け取り、購入が最終販売であることを理解しているにも関わらず、故意に取引に異議を申し立てるシナリオで、裏付け証拠を収集するためのさらなる調査が必要な場合。

これらのシナリオに対処するため、3つの異なるエージェントを導入します：

- **トリアージエージェント:**  
  注文の履行状況に基づいて、紛争を受け入れるか、エスカレーションするかを決定します。

- **受け入れエージェント:**  
  明確なケースを処理し、簡潔な理由付けとともに紛争を自動的に受け入れます。

- **調査エージェント:**  
コミュニケーション記録と注文情報を分析して重要な証拠を収集し、紛争の徹底的な調査を実行します。

このクックブック全体を通して、カスタムエージェントワークフローがどのように紛争管理を自動化し、ビジネス運用をサポートできるかを、段階的にガイドしながら説明します。

## 前提条件

このクックブックを実行する前に、以下のアカウントを設定し、いくつかのセットアップ作業を完了する必要があります。これらの前提条件は、このプロジェクトで使用するAPIと連携するために不可欠です。

#### 1. OpenAIアカウント

- **目的：**  
  このクックブックで紹介されている言語モデルにアクセスし、Agents SDKを使用するためにOpenAIアカウントが必要です。

- **アクション：**  
  まだアカウントをお持ちでない場合は、[OpenAIアカウントにサインアップ](https://openai.com)してください。アカウントを作成したら、[OpenAI API Keysページ](https://platform.openai.com/api-keys)にアクセスしてAPIキーを作成してください。

#### 2. Stripeアカウント

- **目的：**  
  デモワークフローの一部として、決済処理のシミュレーション、紛争の管理、Stripe APIとの連携を行うためにStripeアカウントが必要です。

- **アクション：**  
  [Stripeサインアップページ](https://dashboard.stripe.com/register)にアクセスして無料のStripeアカウントを作成してください。

- **APIキーの場所：**  
  Stripeダッシュボードにログインし、**Developers > API keys**に移動してください。

- **テストモードの使用：**  
  すべての開発とテストには**Test Secret Key**を使用してください。


#### 3. OpenAI APIとStripe APIキーを含む.envファイルの作成

```
OPENAI_API_KEY=
STRIPE_SECRET_KEY=
```

### 環境設定
まず必要な依存関係をインストールし、その後ライブラリをインポートして、後で使用するユーティリティ関数を記述します。

In [ ]:
%pip install python-dotenv --quiet
%pip install openai-agents --quiet
%pip install stripe --quiet
%pip install typing_extensions --quiet

In [211]:
import os
import logging
import json
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool  # Only import what you need
import stripe
from typing_extensions import TypedDict, Any
# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set Stripe API key from environment variables
stripe.api_key = os.getenv("STRIPE_SECRET_KEY")

#### 関数ツールの定義
このセクションでは、紛争処理ワークフローをサポートするいくつかのヘルパー関数ツールを定義します。
<br>
 
- `get_order`、`get_phone_logs`、`get_emails`は、提供された識別子に基づいて注文詳細やメール/電話記録を返すことで、外部データ検索をシミュレートします。
- `retrieve_payment_intent`は、Stripe APIと連携して支払いインテント詳細を取得します。
- `close_dispute`は、提供された紛争IDを使用してStripe紛争を自動的に閉じ、紛争が適切に解決され、ログに記録されることを保証します。

In [212]:
@function_tool
def get_phone_logs(phone_number: str) -> list:
    """
    Return a list of phone call records for the given phone number.
    Each record might include call timestamps, durations, notes, 
    and an associated order_id if applicable.
    """
    phone_logs = [
        {
            "phone_number": "+15551234567",
            "timestamp": "2023-03-14 15:24:00",
            "duration_minutes": 5,
            "notes": "Asked about status of order #1121",
            "order_id": 1121
        },
        {
            "phone_number": "+15551234567",
            "timestamp": "2023-02-28 10:10:00",
            "duration_minutes": 7,
            "notes": "Requested refund for order #1121, I told him we were unable to refund the order because it was final sale",
            "order_id": 1121
        },
        {
            "phone_number": "+15559876543",
            "timestamp": "2023-01-05 09:00:00",
            "duration_minutes": 2,
            "notes": "General inquiry; no specific order mentioned",
            "order_id": None
        },
    ]
    return [
        log for log in phone_logs if log["phone_number"] == phone_number
    ]


@function_tool
def get_order(order_id: int) -> str:
    """
    Retrieve an order by ID from a predefined list of orders.
    Returns the corresponding order object or 'No order found'.
    """
    orders = [
        {
            "order_id": 1234,
            "fulfillment_details": "not_shipped"
        },
        {
            "order_id": 9101,
            "fulfillment_details": "shipped",
            "tracking_info": {
                "carrier": "FedEx",
                "tracking_number": "123456789012"
            },
            "delivery_status": "out for delivery"
        },
        {
            "order_id": 1121,
            "fulfillment_details": "delivered",
            "customer_id": "cus_PZ1234567890",
            "customer_phone": "+15551234567",
            "order_date": "2023-01-01",
            "customer_email": "customer1@example.com",
            "tracking_info": {
                "carrier": "UPS",
                "tracking_number": "1Z999AA10123456784",
                "delivery_status": "delivered"
            },
            "shipping_address": {
                "zip": "10001"
            },
            "tos_acceptance": {
                "date": "2023-01-01",
                "ip": "192.168.1.1"
            }
        }
    ]
    for order in orders:
        if order["order_id"] == order_id:
            return order
    return "No order found"


@function_tool
def get_emails(email: str) -> list:
    """
    Return a list of email records for the given email address.
    """
    emails = [
        {
            "email": "customer1@example.com",
            "subject": "Order #1121",
            "body": "Hey, I know you don't accept refunds but the sneakers don't fit and I'd like a refund"
        },
        {
            "email": "customer2@example.com",
            "subject": "Inquiry about product availability",
            "body": "Hello, I wanted to check if the new model of the smartphone is available in stock."
        },
        {
            "email": "customer3@example.com",
            "subject": "Feedback on recent purchase",
            "body": "Hi, I recently purchased a laptop from your store and I am very satisfied with the product. Keep up the good work!"
        }
    ]
    return [email_data for email_data in emails if email_data["email"] == email]


@function_tool
async def retrieve_payment_intent(payment_intent_id: str) -> dict:
    """
    Retrieve a Stripe payment intent by ID.
    Returns the payment intent object on success or an empty dictionary on failure.
    """
    try:
        return stripe.PaymentIntent.retrieve(payment_intent_id)
    except stripe.error.StripeError as e:
        logger.error(f"Stripe error occurred while retrieving payment intent: {e}")
        return {}

@function_tool
async def close_dispute(dispute_id: str) -> dict:
    """
    Close a Stripe dispute by ID. 
    Returns the dispute object on success or an empty dictionary on failure.
    """
    try:
        return stripe.Dispute.close(dispute_id)
    except stripe.error.StripeError as e:
        logger.error(f"Stripe error occurred while closing dispute: {e}")
        return {}


### エージェントの定義

- **紛争受付エージェント（investigator_agent）**は、すべての関連証拠を収集し、レポートを提供することで紛争を調査する責任を負います。
- **紛争受理エージェント（accept_dispute_agent）**は、有効と判断された紛争を自動的にクローズし、その決定に対する簡潔な説明を提供することで処理します。
- **トリアージエージェント（triage_agent）**は、決定者として機能し、支払いインテントのメタデータから注文IDを抽出し、詳細な注文情報を取得した後、紛争を調査員にエスカレーションするか、紛争受理エージェントに渡すかを決定します。
- これらのエージェントは連携して、特定のタスクを専門化されたエージェントに委任することで、紛争解決プロセスを自動化し、合理化するモジュラーワークフローを形成します。

In [213]:
investigator_agent = Agent(
    name="Dispute Intake Agent",
    instructions=(
        "As a dispute investigator, please compile the following details in your final output:\n\n"
        "Dispute Details:\n"
        "- Dispute ID\n"
        "- Amount\n"
        "- Reason for Dispute\n"
        "- Card Brand\n\n"
        "Payment & Order Details:\n"
        "- Fulfillment status of the order\n"
        "- Shipping carrier and tracking number\n"
        "- Confirmation of TOS acceptance\n\n"
        "Email and Phone Records:\n"
        "- Any relevant email threads (include the full body text)\n"
        "- Any relevant phone logs\n"
    ),
    model="o3-mini",
    tools=[get_emails, get_phone_logs]
)


accept_dispute_agent = Agent(
    name="Accept Dispute Agent",
    instructions=(
        "You are an agent responsible for accepting disputes. Please do the following:\n"
        "1. Use the provided dispute ID to close the dispute.\n"
        "2. Provide a short explanation of why the dispute is being accepted.\n"
        "3. Reference any relevant order details (e.g., unfulfilled order, etc.) retrieved from the database.\n\n"
        "Then, produce your final output in this exact format:\n\n"
        "Dispute Details:\n"
        "- Dispute ID\n"
        "- Amount\n"
        "- Reason for Dispute\n\n"
        "Order Details:\n"
        "- Fulfillment status of the order\n\n"
        "Reasoning for closing the dispute\n"
    ),
    model="gpt-4o",
    tools=[close_dispute]
)

triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "Please do the following:\n"
        "1. Find the order ID from the payment intent's metadata.\n"
        "2. Retrieve detailed information about the order (e.g., shipping status).\n"
        "3. If the order has shipped, escalate this dispute to the investigator agent.\n"
        "4. If the order has not shipped, accept the dispute.\n"
    ),
    model="gpt-4o",
    tools=[retrieve_payment_intent, get_order],
    handoffs=[accept_dispute_agent, investigator_agent],
)


#### 争議を取得してエージェントワークフローを開始する
この関数は、提供された`payment_intent_id`を使用してStripeから争議の詳細を取得し、取得した争議情報を指定された`triage_agent`に渡すことで争議処理ワークフローを開始します。

In [214]:
async def process_dispute(payment_intent_id, triage_agent):
    """Retrieve and process dispute data for a given PaymentIntent."""
    disputes_list = stripe.Dispute.list(payment_intent=payment_intent_id)
    if not disputes_list.data:
        logger.warning("No dispute data found for PaymentIntent: %s", payment_intent_id)
        return None
    
    dispute_data = disputes_list.data[0]
    
    relevant_data = {
        "dispute_id": dispute_data.get("id"),
        "amount": dispute_data.get("amount"),
        "due_by": dispute_data.get("evidence_details", {}).get("due_by"),
        "payment_intent": dispute_data.get("payment_intent"),
        "reason": dispute_data.get("reason"),
        "status": dispute_data.get("status"),
        "card_brand": dispute_data.get("payment_method_details", {}).get("card", {}).get("brand")
    }
    
    event_str = json.dumps(relevant_data)
    # Pass the dispute data to the triage agent
    result = await Runner.run(triage_agent, input=event_str)
    logger.info("WORKFLOW RESULT: %s", result.final_output)
    
    return relevant_data, result.final_output

#### シナリオ1: 会社のミス（商品未受領）
このシナリオは、会社が明らかにエラーを犯した状況を表しています。例えば、注文の履行や発送に失敗した場合などです。このような場合、争議を争うのではなく、受け入れることが適切である可能性があります。

In [ ]:
payment = stripe.PaymentIntent.create(
  amount=2000,
  currency="usd",
  payment_method = "pm_card_createDisputeProductNotReceived",
  confirm=True,
  metadata={"order_id": "1234"},
  off_session=True,
  automatic_payment_methods={"enabled": True},
)
relevant_data, triage_result = await process_dispute(payment.id, triage_agent)

#### シナリオ2: 顧客による異議申し立て（最終販売）
このシナリオは、顧客が正しい商品を受け取り、購入が明確に「最終販売」（返金・返品不可）として表示されていることを十分に認識していたにもかかわらず、意図的に取引に対して異議を申し立てる状況を説明しています。このような異議申し立てには、通常、効果的に争うための証拠を収集するためのさらなる調査が必要です。

In [ ]:
payment = stripe.PaymentIntent.create(
  amount=2000,
  currency="usd",
  payment_method = "pm_card_createDispute",
  confirm=True,
  metadata={"order_id": "1121"},
  off_session=True,
  automatic_payment_methods={"enabled": True},
)
relevant_data, triage_result = await process_dispute(payment.id, triage_agent)

## 結論

このJupyter Notebookでは、**OpenAI Agents SDK**の機能を探求し、シンプルでPythonファーストなアプローチを使用してエージェントベースのAIアプリケーションを効率的に作成する方法を実演しました。具体的には、以下のSDK機能を紹介しました：

- **Agent Loop**: ツール呼び出しを管理し、結果をLLMに伝達し、完了まで繰り返し処理を行います。
- **Handoffs**: 複数の専門エージェント間での調整とタスクの委譲を可能にします。
- **Function Tools**: Pythonの関数を自動スキーマ生成と検証機能付きのツールに変換します。

さらに、SDKはOpenAIダッシュボード経由でアクセス可能な組み込み**トレーシング**機能を提供します。トレーシングは、開発段階と本番段階の両方でエージェントワークフローの可視化、デバッグ、監視を支援します。また、OpenAIの評価、ファインチューニング、蒸留ツールとスムーズに統合されます。

このノートブックでは直接取り上げませんでしたが、本番アプリケーションでは入力の検証とエラーの事前検出のために**Guardrails**の実装を強く推奨します。

全体として、このノートブックはさらなる探求のための明確な基盤を築き、OpenAI Agents SDKがいかに直感的で効果的なエージェント駆動ワークフローを促進するかを強調しています。